In [3]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import ascii, votable
from astropy.table import Table, vstack, hstack
import astropy.units as u
from astropy.coordinates import ICRS, Galactic
from scipy.interpolate import interp1d

from os import listdir

dir_rrlyrae = '/datascope/menard/group/scheng/Gaia/'

## load rrlyrae table

In [4]:
rrlyrae = np.load(dir_rrlyrae+'rrlyrae.npy')[0]['rrlyrae']
len(rrlyrae)

140784

## processing the name of light-curve folders

In [5]:
dir_list = listdir('/datascope/menard/DATA/Gaia/gdr2_new/Gaia/gdr2/light_curves/csv/')
id_list_start = np.zeros(len(dir_list),dtype=int)
id_list_end = np.zeros(len(dir_list),dtype=int)
for i in range(len(dir_list)):
    underscore = []
    dot = []
    for j in range(len(dir_list[i])):
        if dir_list[i][j] == '_':
            underscore.append(j)
        elif dir_list[i][j] == '.':
            dot.append(j)
    id_list_start[i] = dir_list[i][13:underscore[2]]
    id_list_end[i] = dir_list[i][underscore[2]+1:dot[0]]

## sort light curve folder by source id 

In [6]:
file_list = Table([id_list_end, dir_list],names=('id_list_end','dir_list'),dtype=(int,str))
file_list.add_column(Table.Column(np.arange(len(id_list_end)),name='index_unordered',dtype=int))
file_list.sort('id_list_end')
len(dir_list)

551

### sort RR Lyrae table by id

In [7]:
table_id_ordered = rrlyrae.copy()
table_id_ordered.add_column(Table.Column(np.arange(len(table_id_ordered)),name='index',dtype=int))
table_id_ordered.sort('source_id')
table_id_ordered.add_column(Table.Column(np.arange(len(table_id_ordered)),name='inverse_index',dtype=int))
table_id_ordered.sort('index')
inverse_index = table_id_ordered['inverse_index'].copy()
table_id_ordered.sort('source_id')

### select rr lyrae

In [8]:
#selected_ab_18_bright = (table_id_ordered['best_classification']==b'RRab') * (table_id_ordered['int_average_g']<1000)
table_id_ordered_selected = table_id_ordered
len(table_id_ordered)

140784

## Extract RR Lyrae

In [12]:
rrlyrae_index = np.load(dir_rrlyrae+'rrlyrae_lightcurve_index.npy')
light_curve_total = np.load(dir_rrlyrae+'lightcurve_total.npy')[0]['light_curve_total']

len(rrlyrae_index)

In [41]:
selected = np.zeros(len(light_curve_total),dtype=bool)
for i in range(len(rrlyrae_index)):
    selected[rrlyrae_index[i,0]+rrlyrae_index[i,4]:rrlyrae_index[i,1]+rrlyrae_index[i,4]] = True
    selected[rrlyrae_index[i,1]+rrlyrae_index[i,4]:rrlyrae_index[i,2]+rrlyrae_index[i,4]] = True
    selected[rrlyrae_index[i,2]+rrlyrae_index[i,4]:rrlyrae_index[i,3]+rrlyrae_index[i,4]] = True

    selected.sum()/140784, len(selected)

(81.64731787703148, 48255086)

In [42]:
light_curve_rrlyrae = light_curve_total[selected]

In [43]:
np.save(dir_rrlyrae+'lightcurve_rrlyrae.npy',np.array([{'light_curve_rrlyrae':light_curve_rrlyrae}]))

In [73]:
G_length = rrlyrae_index[:,1] - rrlyrae_index[:,0]
BP_length = rrlyrae_index[:,2] - rrlyrae_index[:,1]
RP_length = rrlyrae_index[:,3] - rrlyrae_index[:,2]
total_length = G_length + BP_length + RP_length

(G_length==0).sum()

316

In [74]:
cumulative = total_length.copy()
cumulative[0] = 0
for i in range(1,len(cumulative)):
    cumulative[i] = cumulative[i-1] + total_length[i-1]

In [86]:
rrlyrae_index_new = np.zeros((len(rrlyrae),4),dtype=int) 
for i in range(len(rrlyrae_index)):
    rrlyrae_index_new[i,0] = cumulative[i]
    rrlyrae_index_new[i,1] = cumulative[i] + G_length[i]
    rrlyrae_index_new[i,2] = cumulative[i] + G_length[i] + BP_length[i]
    rrlyrae_index_new[i,3] = cumulative[i] + total_length[i]

## test

In [92]:
i=140783
print(table_id_ordered['source_id'][i])
light_curve_rrlyrae[rrlyrae_index_new[i,0]-1:rrlyrae_index_new[i,2]+1]

6917264079699449600


source_id,transit_id,band,time,mag,flux,flux_error,flux_over_error,rejected_by_photometry,rejected_by_variability,other_flags,solution_id
int64,int64,str2,float64,float64,float64,float64,float64,str5,str5,int64,int64
6917091460669437824,50768691901293058,RP,2314.969855642799,16.051755351648513,3048.357194794994,58.85471764183874,51.79461,false,false,0,369295549951641967
6917264079699449600,20072450219857235,G,1759.8451417789568,19.268282010330086,369.8566961746517,14.856225030971387,24.895739,false,false,4097,369295549951641967
6917264079699449600,20076542511068047,G,1759.9191347005833,19.320326328347328,352.5459733652571,4.144063924094932,85.072525,false,false,524289,369295549951641967
6917264079699449600,21720777228980342,G,1789.651412334692,19.324673943641102,351.13709560502235,4.639141955544111,75.690094,false,false,1,369295549951641967
6917264079699449600,21730518037460592,G,1789.82755338045,18.952301959511043,494.79523229872694,8.265460523321343,59.862995,false,false,1,369295549951641967
6917264079699449600,22850334415418476,G,1810.0771105355218,19.33319209925581,348.39302396701777,8.288440888162961,42.033604,false,false,1,369295549951641967
6917264079699449600,22854426665738884,G,1810.1511110598385,19.293469431280137,361.37534353714665,6.447456292234253,56.04929,false,false,1,369295549951641967
6917264079699449600,29970608288838337,G,1938.845714957021,19.268910342548626,369.6427163462756,7.145272333245077,51.732487,false,false,1,369295549951641967
6917264079699449600,29974700574961931,G,1938.9197288011603,19.332786689390026,348.52313691836076,11.35059907619466,30.705263,false,false,1,369295549951641967


## save

In [94]:
np.save(dir_rrlyrae+'rrlyrae_lightcurve_index_new.npy',rrlyrae_index_new)